In [1]:
import json
import numpy as np
import os
import re

In [2]:
cd ../..

C:\Users\14102\python_projects\JohnStreetProjects


In [11]:
def find_ngrams(word, n) -> list:
    return list(zip(*[word[i:] for i in range(n)]))

## Testing out train json parsing

Wont work without the entire spider dataset downloaded.

In [ ]:
train_file_path = 'spider/spider/train_spider.json'
with open(train_file_path) as f:
    data_dict = json.load(f)

In [ ]:
racetrack_qrys = [dct for dct in data_dict if dct['db_id'] == 'race_track']

In [ ]:
with open('racetrack_queries.json', 'w') as f:
    json.dump(racetrack_qrys, f)

## Testing out schema manipulation

In [3]:
db_dir = 'txt2sql/Schema_Processing'
os.listdir(db_dir)

['.ipynb_checkpoints',
 'Preprocess_experiment.ipynb',
 'racetrack_queries.json',
 'race_track']

In [4]:
with open(db_dir+'/race_track/racetrack_schema.sql') as f:
    data = f.read()

In [5]:
data = data.replace('\n', '').replace(' ', '').replace('`', "\"").upper().split(';')

In [6]:
data

['PRAGMAFOREIGN_KEYS=ON',
 'CREATETABLE"RACE"("RACE_ID"INT,"NAME"TEXT,"CLASS"TEXT,"DATE"TEXT,"TRACK_ID"TEXT,PRIMARYKEY("RACE_ID"),FOREIGNKEY("TRACK_ID")REFERENCES"TRACK"("TRACK_ID"))',
 'CREATETABLE"TRACK"("TRACK_ID"INT,"NAME"TEXT,"LOCATION"TEXT,"SEATING"REAL,"YEAR_OPENED"REAL,PRIMARYKEY("TRACK_ID"))',
 'INSERTINTO"TRACK"VALUES(1,"AUTOCLUBSPEEDWAY","FONTANA,CA","92000","1997")',
 'INSERTINTO"TRACK"VALUES(2,"CHICAGOLANDSPEEDWAY","JOLIET,IL","75000","2001")',
 'INSERTINTO"TRACK"VALUES(3,"DARLINGTONRACEWAY","DARLINGTON,SC","63000","1950")',
 'INSERTINTO"TRACK"VALUES(4,"DAYTONAINTERNATIONALSPEEDWAY","DAYTONABEACH,FL","168000","1959")',
 'INSERTINTO"TRACK"VALUES(5,"HOMESTEAD-MIAMISPEEDWAY","HOMESTEAD,FL","65000","1995")',
 'INSERTINTO"TRACK"VALUES(6,"KANSASSPEEDWAY","KANSASCITY,KS","81687","2001")',
 'INSERTINTO"TRACK"VALUES(7,"MARTINSVILLESPEEDWAY","RIDGEWAY,VA","65000","1947")',
 'INSERTINTO"TRACK"VALUES(8,"MICHIGANINTERNATIONALSPEEDWAY","BROOKLYN,MI","137243","1968")',
 'INSERTINTO"TRACK

In [7]:
# Define regular expressions patterns
table_pattern = r'CREATETABLE"(.*?)"'
column_pattern = r'"(.+?)"([\w\(\)]+)[,$]'
primary_key_pattern = r'PRIMARYKEY\("(.*?)"\)'
foreign_key_pattern = r'FOREIGNKEY\("(.*?)"\)REFERENCES"(.*?)"\("(.*?)"\)'
schema = {}

elem_qty = 0
for table_def in data:
    if re.search(r"^CREATETABLE", table_def):
        elem_qty += 1
        # Extract table name
        table_name = re.search(table_pattern, table_def).group(1)
        table_def = re.sub(table_pattern, '', table_def)
        # Extract columns
        columns = re.findall(column_pattern, table_def)
        elem_qty += len(columns)
        # Extract primary key
        primary_keys = re.findall(r'PRIMARYKEY\((.*?)\)', table_def)
        primary_keys = re.findall(r'"(\w+)"', primary_keys[0]) if primary_keys else []

#         primary_key = re.search(primary_key_pattern, table_def)
#         primary_key_column = primary_key.group(1) if primary_key else None

        # Extract foreign keys
        foreign_keys = re.findall(foreign_key_pattern, table_def)
        
        col_rels = {}
        for col in columns:
            fks = [fk[1:] for fk in foreign_keys if col[0] == fk[0]]
            col_rels[col[0]] = {'fks': fks, 'ispk': col[0] in primary_keys}
        schema[table_name] = col_rels
        # Display extracted data
#         print("Table Name:", table_name)
#         print("Columns:", columns)
#         print("Primary Key:", primary_key_column)
#         print("Foreign Keys:", foreign_keys)
schema

{'RACE': {'RACE_ID': {'fks': [], 'ispk': True},
  'NAME': {'fks': [], 'ispk': False},
  'CLASS': {'fks': [], 'ispk': False},
  'DATE': {'fks': [], 'ispk': False},
  'TRACK_ID': {'fks': [('TRACK', 'TRACK_ID')], 'ispk': False}},
 'TRACK': {'TRACK_ID': {'fks': [], 'ispk': True},
  'NAME': {'fks': [], 'ispk': False},
  'LOCATION': {'fks': [], 'ispk': False},
  'SEATING': {'fks': [], 'ispk': False},
  'YEAR_OPENED': {'fks': [], 'ispk': False}}}

In [8]:
# rel_matrix = np.zeros((elem_qty, elem_qty))
# cur_idx = 0
# for table_name, col_dict in schema.items():
    
#     for col_id, (col_name, col_rels) in enumerate(col_dict.items()):
#         print(col_id)

In [53]:
qry = 'show me race on all track'.upper()
qry_ngrams = set()
for ngram_size in range(5):
    qry_ngrams.update(set(find_ngrams(qry.split(' '), ngram_size + 1)))
qry_ngram_list = list(qry_ngrams)

In [54]:
entity_to_index

{'RACE': 0,
 ('RACE', 'RACE_ID'): 1,
 ('RACE', 'NAME'): 2,
 ('RACE', 'CLASS'): 3,
 ('RACE', 'DATE'): 4,
 ('RACE', 'TRACK_ID'): 5,
 'TRACK': 6,
 ('TRACK', 'TRACK_ID'): 7,
 ('TRACK', 'NAME'): 8,
 ('TRACK', 'LOCATION'): 9,
 ('TRACK', 'SEATING'): 10,
 ('TRACK', 'YEAR_OPENED'): 11,
 ('RACES', 'ON'): 12,
 ('SHOW', 'ME'): 13,
 ('ME', 'RACES'): 14,
 ('SHOW',): 15,
 ('SHOW', 'ME', 'RACES'): 16,
 ('ALL',): 17,
 ('ON', 'ALL'): 18,
 ('RACES', 'ON', 'ALL'): 19,
 ('SHOW', 'ME', 'RACES', 'ON', 'ALL'): 20,
 ('RACES',): 21,
 ('ME', 'RACES', 'ON'): 22,
 ('TRACKS',): 23,
 ('ME', 'RACES', 'ON', 'ALL', 'TRACKS'): 24,
 ('RACES', 'ON', 'ALL', 'TRACKS'): 25,
 ('ME', 'RACES', 'ON', 'ALL'): 26,
 ('ON',): 27,
 ('ME',): 28,
 ('ALL', 'TRACKS'): 29,
 ('SHOW', 'ME', 'RACES', 'ON'): 30,
 ('ON', 'ALL', 'TRACKS'): 31}

In [56]:
import numpy as np

# Sample schema data
schema_data = schema

# Get all table names
table_names = list(schema_data.keys())

# Create a matrix to represent relationships between columns and tables
num_entities = len(table_names) + sum(len(schema_data[table]) for table in table_names) + len(qry_ngrams)
relationship_matrix = np.zeros((num_entities, num_entities), dtype=object)

# Map indices to column names, table names, and their types (column or table)
entity_to_index = {}
index = 0

# Map column indices to column names, table names, and their types
for table in table_names:
    entity_to_index[table] = index
    index += 1
    for column in schema_data[table]:
        entity_to_index[(table, column)] = index
        index += 1

# Map ngrams parsed from NLQ to matrix indices
for ngram in qry_ngram_list:
    entity_to_index[ngram] = index
    index += 1


# Update matrix based on relationships (Primary keys, Foreign keys, Column membership)
for table in table_names:
    table_index = entity_to_index[table]
    for column, details in schema_data[table].items():
        column_index = entity_to_index[(table, column)]
        if details['ispk']:
            # Mark column as primary key
            relationship_matrix[column_index, table_index] = 'PKOf'
            relationship_matrix[table_index, column_index] = 'HasPK'
        else:
            # Mark column as a member of a table
#             relationship_matrix[column_index, column_index] = 'Member'
            # Mark relationship between column and table
            relationship_matrix[column_index, table_index] = 'ColInTbl'
            relationship_matrix[table_index, column_index] = 'TblOfCol'
        for fk_table, fk_column in details['fks']:
            # Mark column as a foreign key pointing to another column
            fk_column_index = entity_to_index[(fk_table, fk_column)]
            relationship_matrix[column_index, fk_column_index] = 'ColFKDepOn'
            relationship_matrix[column_index, entity_to_index[fk_table]] = 'TblFKDepIn'
            relationship_matrix[fk_column_index, column_index] = 'ColFKDepFrom'
            relationship_matrix[entity_to_index[fk_table], column_index] = 'TblFKDepFrom'
        
        for qry_ngram in qry_ngram_list:
            qn_id = entity_to_index[qry_ngram]
            if all([seq == column for seq in qry_ngram]):
                relationship_matrix[column_index, qn_id] = 'Col_qstn_full_match'
                relationship_matrix[qn_id, column_index] = 'Qstn_col_full_match'
                
            elif any([seq == column for seq in qry_ngram]):
                relationship_matrix[column_index, qn_id] = 'Col_qstn_partial_match'
                relationship_matrix[qn_id, column_index] = 'Qstn_col_partial_match'
            else:
                relationship_matrix[column_index, qn_id] = 'Col_qstn_no_match'
                relationship_matrix[qn_id, column_index] = 'Qstn_col_no_match'
                
    for qry_ngram in qry_ngram_list:
        qn_id = entity_to_index[qry_ngram]
        if all([seq == table for seq in qry_ngram]):
            relationship_matrix[table_index, qn_id] = 'Tbl_qstn_full_match'
            relationship_matrix[qn_id, table_index] = 'Qstn_tbl_full_match'
            
        elif any([seq == table for seq in qry_ngram]):
            relationship_matrix[table_index, qn_id] = 'Tbl_qstn_partial_match'
            relationship_matrix[qn_id, table_index] = 'Qstn_tbl_partial_match'
        else:
            relationship_matrix[table_index, qn_id] = 'tbl_qstn_no_match'
            relationship_matrix[qn_id, table_index] = 'Qstn_tbl_no_match'

# Display the relationship matrix
# for i in range(num_entities):
#     for j in range(num_entities):
#         print(relationship_matrix[i, j], end="\t")
#     print()
relationship_matrix

array([[0, 'HasPK', 'TblOfCol', ..., 'Tbl_qstn_partial_match',
        'Tbl_qstn_partial_match', 'tbl_qstn_no_match'],
       ['PKOf', 0, 0, ..., 'Col_qstn_no_match', 'Col_qstn_no_match',
        'Col_qstn_no_match'],
       ['ColInTbl', 0, 0, ..., 'Col_qstn_no_match', 'Col_qstn_no_match',
        'Col_qstn_no_match'],
       ...,
       ['Qstn_tbl_partial_match', 'Qstn_col_no_match',
        'Qstn_col_no_match', ..., 0, 0, 0],
       ['Qstn_tbl_partial_match', 'Qstn_col_no_match',
        'Qstn_col_no_match', ..., 0, 0, 0],
       ['Qstn_tbl_no_match', 'Qstn_col_no_match', 'Qstn_col_no_match',
        ..., 0, 0, 0]], dtype=object)